In [3]:
import numpy as np
import random
from numpy import inf


In [34]:
cost_matrix = np.array([[inf, 1,   inf, inf, inf],
                        [2,   inf, 3,   1,   inf],
                        [inf, 2,   inf, 1,   inf],
                        [inf, 1,   inf, inf, 2  ],
                        [inf, inf, 2,   inf, inf]])

starting_points = [0,4]

number_of_points = len(cost_matrix)

start_pos=[0,4]

number_of_solutions = 100
solutions = []
solutions_scores = []

def generate_random_solution(number_of_points):
    result = list(range(number_of_points))
    random.shuffle(result)
    return np.array(result)
    

# print(generate_random_solution(number_of_points))

def eval_solution(path):
    time_multiplier = 1
    missed_path_multiplier = 1000
    out_of_bounds_multiplier = 10000

    fitness = 0

    # czas przejścia
    for i in range(len(path)-1):
        fitness += time_multiplier*cost_matrix[path[i],path[i+1]]

    # kara za pominięcie szlaku
    all_paths = np.ma.masked_invalid(cost_matrix).nonzero()  # TODO wyjebać to poza funkcje
    all_paths = list(zip(*all_paths)) # transpose
    all_paths = list({*map(tuple, map(sorted, all_paths))}) # remove duplicates

    print(all_paths)

    number_of_missed_paths = 0
    for a,b in all_paths:
        if (repr(path).count(f"{a}, {b}") == 0 and repr(path).count(f"{b}, {a}") == 0):
            number_of_missed_paths+=1

    fitness += missed_path_multiplier*number_of_missed_paths

    # kara za wyjście poza szlak
    number_of_oob_paths = 0
    for i in range(len(path)-1):
        if (cost_matrix[path[i], path[i+1]]==inf):
            number_of_oob_paths += 1

    fitness += out_of_bounds_multiplier*number_of_oob_paths

    return fitness

# print(eval_solution([0,1,2,3,4]))


def generate_neighbours(path, number_of_points):
    rand = random.randint(1, 3) 
    position = random.randint(0, len(path))
    position2 = random.randint(0, len(path))
    new_value = random.randint(0, number_of_points-1)
    if rand == 1:
        path.append(new_value)

    elif rand ==2:
        path[position], path[position2] = path[position2], path[position]

    else:
        path[position]=new_value

for i in range(number_of_solutions):
    solutions.append(generate_random_solution(number_of_points))
    